<h1> Excersises 8 - Programming continues </h1>

<h3> Modyfing data </h3>

Lastly we have loaded data layers, did selection on them and we accessed attributes of vector and raster layers. 

Today we will edit parameters and values of object of vector layers.

We will work on exemplay vector layer. Reminder: to access currently loaded layer:

In [ ]:
#Has to be selected in a layer list window!
warstwa = iface.activeLayer()

Changes and modifications are not visible until refreshed. To refresh a layer to see changes on the map canvas:

In [ ]:
#refreshibg
warstwa.triggerRepaint() #by layer
#OR
iface.mapCanvas().refresh() #by interface

<h3> Adding new objects </h3>

We assume working on a vector layer with LineString type of data (lines). Such a file was attachet to excersise files.

Each object (QgsFeature) of the vector layer (QgsVectorLayer) has:
<ul>
    <li> Attributes (QgsField) </li>
    <li> Geometry (QgsGeometry) </li>
</ul>

When creating new object both have to be set.

First we create an empty new object:

In [ ]:
feat = QgsFeature(warstwa.fields()) #initilizing new QgsFeature object, fields of a layer are the parametres

It is not assigned to the layer yet, we will do it later. First it has to be given attributes' values and geometry:

In [ ]:
feat.setAttributes([0, 60]) #adding attributes, this layer has to attributes of int type
#OR
feat.setAttribute(0,60) #if we are adding only one row

Now we create geometry:

In [ ]:
#Line from two points. First we create two objects of QgsPoint type - see description below:
line_start = QgsPoint(15.74,50.89)
line_end = QgsPoint(15.73,50.91)

QgsGeometry is a class of geometry, which can be points, line or polygons. Most oftenly geometry is build from series of points (either points, lines or polygons). For that we use([new class here!]) QgsPoint:

    QgsPoint(x,y)

After, we use on of the methods that create QgsGeometry from QgsPoints .from __geometria__, that is:
 
    fromPolyline, fromPolygonXY, fromPointXY or fromWKT - that is WKT type of data

In [ ]:
feat.setGeometry(QgsGeometry.fromPolyline([line_start,line_end]))

And we add it to the layer now. For that we are sing the dataProvider class, which is always used for all the operations of adding, deleting and editing objects of a layer (more about it below):

In [ ]:
warstwa.dataProvider().addFeatures([feat]) #wee add the feature to the layer

Besides QgsPoints, geometry can be build from WKT, which is much easier, because EVERY type of can be written in that file format.

<h3> WKT - Well-known text representation of geometry </h3>

It allows to define many types of geometry, including:

    Point (single), MultiPoint (collection of Points),
    LineString (single), MultiLineString (collection of Lines),
    Polygon (singlle), MultiPolygon (collection of Polygons).

Definition of a geometry:

    Nazwa (x y)

Point:

    POINT (10 30) - Point with coordinates x=10 i y = 30

Line:

    LINESTRING (30 10, 10 30) - a simple straight line of coordinates of first vertex: X = 30, Y = 10 second vertex: X = 10, Y = 30

    LINESTRING (30 10, 10 30, 40 40) - line consisting of three points.
    
Polygon:

    POLYGON ((30 10, 40 40, 20 20, 10 20, 30 10))
    
    POLYGON ((35 10, 45 45, 15 40, 10 20, 35 10), (20 30, 35 35, 30 20, 20 30))
    
    The first and the last points is repeating because polygon has to has close geometry (so the example has three vertices.
    
    Double pharenteses allow for adding a "hole" in the geometry (as in the second example).
    

Other examples are pretty well-explained on Wikipedia:

    https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
    
<h3> WKB - Well-known binary </h3>

Binary equivalent of WKT. Faster for reading and writing, but not readable.

<b> Excersise: </b> Try to realize above geometry (of lineString) using WKT. WKT is written as a text " ". Use command .fromWKT()